In [65]:
import pandas as pd
import numpy as np

import sklearn.cluster
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn import datasets

import sklearn.metrics
from sklearn.linear_model import LogisticRegression

In [83]:
# Função geradora da função de ativação do kernel
def make_gaussian_kernel(center, sigma):
    variance = sigma**2
    gamma = 2*(variance)
    reshaped_center = np.reshape(center, newshape = (1,-1))
    def gaussian(x):
        dist = euclidean_distances(x, reshaped_center, squared=True)
        normalization_constant = 1/(2*np.pi*variance)
        return  normalization_constant * np.exp(-(dist/gamma))
    return gaussian

In [66]:
# Inputs
seed = 42
k = 6

In [68]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [70]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 50, random_state = seed)

In [71]:
kmeans = sklearn.cluster.KMeans(n_clusters = k, random_state = seed)

In [72]:
kmeans.fit(X_treino)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=6, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=42, tol=0.0001, verbose=0)

In [73]:
centros = kmeans.cluster_centers_ 
# pd.DataFrame(centros)

In [74]:
classificados = kmeans.predict(X_treino)
# classificados

In [82]:
# Calculando os desvios de cada cluster
distancias_para_cada_centro = euclidean_distances(X_treino, centros)

df_distancias = pd.DataFrame(distancias_para_cada_centro)
df_distancias['classe'] = classificados

distancias_medias = df_distancias.groupby('classe').mean()
distancias_desvios = df_distancias.groupby('classe').std()

medias_por_grupo = np.diag(distancias_medias)
desvios_por_grupo = np.diag(distancias_desvios)

In [16]:
# Obtendo as funções radiais
kernels = list()

for cluster_alvo in range(k):
    alvos = X_treino[classificados==cluster_alvo]
    kernel = make_gaussian_kernel(centros[cluster_alvo], desvios_por_grupo[cluster_alvo])
    kernels.append(kernel)

In [53]:
# Transformando o espaço de entrada
features = [ kernel(X_treino) for kernel in kernels ]
transformed_input = pd.DataFrame(np.concatenate(features, axis = 1))

In [268]:
# Executando a regressão logística
regression = LogisticRegression()
regression.fit(transformed_input, y_treino)


In [279]:
# Output 
predicoes = regression.predict(transformed_input)